In [1]:
import time #sleep함수를 통해 딜레이를 줄 수 있음
import random #적들이 무작위로 나올 때 쓸 수 있음
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction #라즈베리에서 제공하는 공식 라이브러리
from PIL import Image, ImageDraw, ImageFont #파이썬 이미지 처리 라이브러리
from adafruit_rgb_display import st7789 #조이스틱 모듈을 쓰기 위함
import numpy as np

In [2]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000 #조이스틱 핀에 대한 기본적 정의..
        
        self.spi = board.SPI() #SPI 사용
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240, #화면 픽셀 크기가 240X240, 여기 맞춰서 그림을 넣어야한다
                    y_offset=80,
                    rotation=180, #조이스틱 방향을 설정
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )
        # Input pins:
        self.button_A = DigitalInOut(board.D5) #버튼 A
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6) #버튼 B
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27) #왼쪽 방향으로
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23) #오른쪽 방향으로
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17) #위로
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22) #아래로
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4) #가운데 버튼
        self.button_C.direction = Direction.INPUT
        
        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True #화면에 불이 들어옴
        
         # Create blank image for drawing.
        # Make sure to create image with mode 'RGB' for color.
        self.width = self.disp.width
        self.height = self.disp.height #화면 설정? 기본적

In [3]:
class Character:
    def __init__(self, width, height):
        self.appearance = 'circle'
        self.state = None
        self.position = np.array([width/8 - 10, height/8 - 20, width/8 + 10, height/8 + 20])
        self.outline = "#FFFFFF"

    def move(self, command = None):
        if command['move'] == False:
            self.state = None
            self.outline = "#FFFFFF" #검정색상 코드!
        
        else:
            self.state = 'move'
            self.outline = "#FF0000" #빨강색상 코드!

            if command['up_pressed']:
                self.position[1] -= 5
                self.position[3] -= 5

            if command['down_pressed']:
                self.position[1] += 5
                self.position[3] += 5

            if command['left_pressed']:
                self.position[0] -= 5
                self.position[2] -= 5
                
            if command['right_pressed']:
                self.position[0] += 5
                self.position[2] += 5
    '''           
    def collision_check(self, enemys):
        for enemy in enemys:
            collision = self.overlap(self.position, enemy.position)
            
            if collision:
                enemy.state = 'die'
                self.state = 'hit'
    '''

In [4]:
class Miro1:
    def __init__(self, width, height):
        self.appearance = 'miro1'
        #self.state = None
        self.position = np.array([0, height/4 - 10, width/4 + 10, height/4 + 10])
        # 총알 발사를 위한 캐릭터 중앙 점 추가
        # self.center = np.array([(self.position[0] + self.position[2]) - 10, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"
        
    def notdown(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        d=1
        if other_position[3] > ego_position[1] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1]:
            d=0
        return d
    
    def notleft(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        l=1
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=0
        return l
    
    def notup(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        u=1
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=0
        return u

In [5]:
class heartCard:
    def __init__(self, width, height):
        '''
        position : 총알을 발사한 Character의 중앙 위치
        command : 조이스틱 입력 값
        '''
        self.appearance = 'heartcard'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([width/4-8, height/4-10, width/4+8, height/4+14])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
    
    def notdown(self, ego_position, other_position):
        
        d=0
        if other_position[3] > ego_position[1] and other_position[2] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] :
            d=1
            return d
    
    def notright(self, ego_position, other_position):
        
        r=0
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=1
            return r
    
    def notleft(self, ego_position, other_position):
        l=0
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=1
            return l
    
    def notup(self, ego_position, other_position):
        u=0
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=1
            return u
        
# 잔상이 남지 않는 코드 & 대각선 이동 가능

In [6]:
class heartCard2:
    def __init__(self, width, height):
        
        self.appearance = 'heartcard2'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([width/4-10, height-height/4-6, width/4+14, height-height/4+10])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
    
    def notdown(self, ego_position, other_position):
        
        d=0
        if other_position[3] > ego_position[1] and other_position[2] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] :
            d=1
            return d
    
    def notright(self, ego_position, other_position):
        
        r=0
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=1
            return r
    
    def notleft(self, ego_position, other_position):
        l=0
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=1
            return l
    
    def notup(self, ego_position, other_position):
        u=0
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=1
            return u

In [7]:
class Bottle:
    def __init__(self, width, height):
        
        self.appearance = 'bottle'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([width-width/4+25, height/4+13, width-15, height/2-13])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
    
    def notdown(self, ego_position, other_position):
        
        d=0
        if other_position[3] > ego_position[1] and other_position[2] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] :
            d=1
            return d
    
    def notright(self, ego_position, other_position):
        
        r=0
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=1
            return r
    
    def notleft(self, ego_position, other_position):
        l=0
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=1
            return l
    
    def notup(self, ego_position, other_position):
        u=0
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=1
            return u

In [8]:
class Miro2:
    def __init__(self, width, height):
        self.appearance = 'miro2'
        #self.state = None
        self.position = np.array([width/2-10, 0, width/2 + 10, height/4 + 10])
        # 총알 발사를 위한 캐릭터 중앙 점 추가
        self.outline = "#FFFFFF"
    
    def notleft(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        l=1
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=0
        return l
    
    def notright(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        r=1
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=0
        return r
    
    def notup(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        u=1
        if other_position[1] < ego_position[3] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3] and other_position[2] > ego_position[0]:
            u=0
        return u

In [9]:
class Miro3:
    def __init__(self, width, height):
        self.appearance = 'miro3'
        #self.state = None
        self.position = np.array([width-width/4-10, height/4-10, width-width/4 + 10, height/2 - 10])
        # 총알 발사를 위한 캐릭터 중앙 점 추가
        self.outline = "#FFFFFF"
    
    def notleft(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        l=1
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=0
        return l
    
    def notright(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        r=1
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=0
        return r
    
    def notdown(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        d=1
        if other_position[3] > ego_position[1] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] and other_position[2] > ego_position[0]:
            d=0
        return d
    

In [10]:
class Miro4:
    def __init__(self, width, height):
        self.appearance = 'miro4'
        #self.state = None
        self.position = np.array([10, height/2 - 10, width, height/2 + 10])
        self.outline = "#FFFFFF"
        
    def notdown(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        d=1
        if other_position[3] > ego_position[1] and other_position[2] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] :
            d=0
        return d
    
    def notright(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        r=1
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=0
        return r
    
    def notup(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        u=1
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=0
        return u

In [11]:
class Miro5:
    def __init__(self, width, height):
        self.appearance = 'miro5'
        #self.state = None
        self.position = np.array([width/4-10, height-height/4 - 5, width-width/4+10, height-height/4+15])
        self.outline = "#FFFFFF"
        
    def notdown(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        d=1
        if other_position[3] > ego_position[1] and other_position[2] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] :
            d=0
        return d
    
    def notright(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        r=1
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=0
        return r
    
    def notleft(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        l=1
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=0
        return l
    
    def notup(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        u=1
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=0
        return u

In [12]:
class Miro6:
    def __init__(self, width, height):
        self.appearance = 'miro6'
        #self.state = None
        self.position = np.array([width/4-10, height-height/4-5, width/4+10, height])
        # 총알 발사를 위한 캐릭터 중앙 점 추가
        self.outline = "#FFFFFF"
    
    def notleft(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        l=1
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=0
        return l
    
    def notright(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        r=1
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=0
        return r
    
    def notdown(self, ego_position, other_position):
        '''
        두개의 사각형(bullet position, enemy position)이 겹치는지 확인하는 함수
        좌표 표현 : [x1, y1, x2, y2]
        
        return :
            True : if overlap
            False : if not overlap
            
        '''
        d=1
        if other_position[3] > ego_position[1] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] and other_position[2] > ego_position[0]:
            d=0
        return d

In [13]:
class Mushroom:
    def __init__(self, width, height):
        
        self.appearance = 'mushroom'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([10, height-height/4+25, width/4-20, height-15])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
    
    def notdown(self, ego_position, other_position):
        
        d=0
        if other_position[3] > ego_position[1] and other_position[2] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] :
            d=1
            return d
    
    def notright(self, ego_position, other_position):
        
        r=0
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=1
            return r
    
    def notleft(self, ego_position, other_position):
        l=0
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=1
            return l
    
    def notup(self, ego_position, other_position):
        u=0
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=1
            return u

In [ ]:
class Hole1:
    def __init__(self, width, height):
        
        self.appearance = 'hole1'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([width/4+15, height-height/4+15, width/2-15, 15])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
    
    def notdown(self, ego_position, other_position):
        
        d=0
        if other_position[3] > ego_position[1] and other_position[2] > ego_position[0] and other_position[0] < ego_position[2] and other_position[1] < ego_position[1] :
            d=1
            return d
    
    def notright(self, ego_position, other_position):
        
        r=0
        if other_position[2] > ego_position[0] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[0] < ego_position[0]:
            r=1
            return r
    
    def notleft(self, ego_position, other_position):
        l=0
        if other_position[0] < ego_position[2] and other_position[1] < ego_position[3] and other_position[3] > ego_position[1] and other_position[2] > ego_position[2]:
            l=1
            return l
    
    def notup(self, ego_position, other_position):
        u=0
        if other_position[1] < ego_position[3] and other_position[0] > ego_position[0] and other_position[0] < ego_position[2] and other_position[3] > ego_position[3]:
            u=1
            return u

In [ ]:
class Life1:
    def __init__(self):
        
        self.appearance = 'life1'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([135, 5, 105, 35])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
        
class Life2:
    def __init__(self):
        
        self.appearance = 'life2'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([100, 5, 70, 35])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
        
class Life3:
    def __init__(self):
        
        self.appearance = 'life3'
        self.sleep=20
        #self.speed = 20
        #self.damage = 10
        self.position = np.array([65, 5, 35, 35])
        #self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        #self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"

In [ ]:
'''
class HeartCard:
    def __init__(self, position, command):
        
        position : 총알을 발사한 Character의 중앙 위치
        command : 조이스틱 입력 값
        
        self.appearance = 'rectangle'
        self.speed = 20
        self.damage = 10
        self.position = np.array([position[0]-3, position[1]-3, position[0]+3, position[1]+3])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])

        self.direction = {'up' : False, 'down' : False, 'left' : False, 'right' : False}
        self.state = None
        self.outline = "#0000FF"
        
        if command['up_pressed']:
            self.direction['up'] = True
        if command['down_pressed']:
            self.direction['down'] = True
        if command['right_pressed']:
            self.direction['right'] = True
        if command['left_pressed']:
            self.direction['left'] = True

    def move(self):
        if self.direction['up']:
            self.position[1] -= self.speed
            self.position[3] -= self.speed

        if self.direction['down']:
            self.position[1] += self.speed
            self.position[3] += self.speed

        if self.direction['left']:
            self.position[0] -= self.speed
            self.position[2] -= self.speed
            
        if self.direction['right']:
            self.position[0] += self.speed
            self.position[2] += self.speed
    '''

In [14]:
joystick = Joystick() #조이스틱 객체 선언
my_image = Image.new('RGB',(joystick.width, joystick.height))
my_draw = ImageDraw.Draw(my_image,'RGBA') #도화지와 도구를 연결 

In [ ]:
iintro=Image.open('intro.png')
start=0
while True:
    if not joystick.button_A.value:
        break
    my_image.paste(iintro,(0,0))
    joystick.disp.image(my_image)

In [15]:
iground=Image.open('ground.png')
iheartcard=Image.open('heartcard.png')
iheartcard2=Image.open('heartcard2.png')
ibottle=Image.open('bottle.png')
imiro1=Image.open('miro1.png')
imiro2=Image.open('miro2.png')
imiro3=Image.open('miro3.png')
imiro4=Image.open('miro4.png')
imiro5=Image.open('miro5.png')
imiro6=Image.open('miro6.png')
relice=Image.open('elicer.png')
lelice=Image.open('elicel.png')
imushroom=Image.open('mushroom.png')
ilife=Image.open('life.png')


In [19]:
# 잔상이 남지 않는 코드 & 대각선 이동 가능
my_circle = Character(joystick.width, joystick.height)
miro1 = Miro1(joystick.width, joystick.height)
miro2 = Miro2(joystick.width, joystick.height)
miro3 = Miro3(joystick.width, joystick.height)
miro4 = Miro4(joystick.width, joystick.height)
miro5 = Miro5(joystick.width, joystick.height)
miro6 = Miro6(joystick.width, joystick.height)
heartcard = heartCard(joystick.width, joystick.height)
heartcard2 = heartCard2(joystick.width, joystick.height)
bottle = Bottle(joystick.width, joystick.height)
mushroom = Mushroom(joystick.width, joystick.height)
hole1 = Hole1(joystick.width, joystick.height)
life1 = Life1()
life2 = Life2()
life3 = Life3()
'''
enemy_1 = Enemy((50, 50))
enemy_2 = Enemy((200, 200))
enemy_3 = Enemy((150, 50))

enemys_list = [enemy_1, enemy_2, enemy_3]


bullets = []
'''
heartcardslp=0
heartcardslp2=0
heartcrash=30
heartcrash2=30
bottleeat=0
mushroomeat=0
delice=0
lifenum=3
while True:
    command = {'move': False, 'up_pressed': False , 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
    
    if not joystick.button_U.value:  # up pressed
        if miro1.notup(miro1.position, my_circle.position) and miro2.notup(miro2.position, my_circle.position) and miro4.notup(miro4.position, my_circle.position) and miro5.notup(miro5.position, my_circle.position):
            command['up_pressed'] = True
            command['move'] = True
            
    
    if not joystick.button_D.value:  # down pressed
        if miro1.notdown(miro1.position, my_circle.position) and miro3.notdown(miro3.position, my_circle.position) and miro4.notdown(miro4.position, my_circle.position) and miro5.notdown(miro5.position, my_circle.position) and miro6.notdown(miro6.position, my_circle.position):
            command['down_pressed'] = True
            command['move'] = True

    if not joystick.button_L.value: 
        delice=1
        if miro1.notleft(miro1.position, my_circle.position) and miro2.notleft(miro2.position, my_circle.position) and miro3.notleft(miro3.position, my_circle.position) and miro5.notleft(miro5.position, my_circle.position) and miro6.notleft(miro6.position, my_circle.position):
            command['left_pressed'] = True
            command['move'] = True
            

    if not joystick.button_R.value: 
        delice=0
        if miro2.notright(miro2.position, my_circle.position) and miro3.notright(miro3.position, my_circle.position) and miro4.notright(miro4.position, my_circle.position) and miro5.notright(miro5.position, my_circle.position) and miro6.notright(miro6.position, my_circle.position):
            command['right_pressed'] = True
            command['move'] = True
            
    '''
    if not joystick.button_A.value: # A pressed
        bullet = Bullet(my_circle.center, command)
        bullets.append(bullet)
    '''
    
        
    if (heartcard.notup(heartcard.position,my_circle.position) or heartcard.notdown(heartcard.position,my_circle.position) or heartcard.notleft(heartcard.position,my_circle.position) or heartcard.notright(heartcard.position,my_circle.position)) and heartcrash!=0:
        heartcrash-=1
        if heartcrash==29:
            lifenum-=1
    elif (heartcard2.notup(heartcard2.position,my_circle.position) or heartcard2.notdown(heartcard2.position,my_circle.position) or heartcard2.notleft(heartcard2.position,my_circle.position) or heartcard2.notright(heartcard2.position,my_circle.position)) and heartcrash2!=0:
        heartcrash2-=1
        if heartcrash2==29:
            lifenum-=1
        if heartcard.position[3]==joystick.width/2+10:
            heartcardslp=1
            heartcard.position[0]=joystick.width/4-8
            heartcard.position[1]=joystick.height/4-1
            heartcard.position[2]=joystick.width/4+8
            heartcard.position[3]=joystick.height/4
                    
        else:
            if heartcardslp==0:
                heartcard.position[1]+=2
                heartcard.position[3]+=2
            else:
                if heartcard.sleep==0:
                    heartcard.position[1]=joystick.height/4-10
                    heartcard.position[3]=joystick.height/4+14
                    heartcardslp=0
                    heartcard.sleep=20
                else:
                    heartcard.sleep-=1
    else:
        my_circle.move(command)
        
        if heartcard2.position[2]==0:
            heartcardslp2=1
            heartcard2.position[0]=joystick.width/4-1
            heartcard2.position[1]=joystick.height-joystick.height/4-6
            heartcard2.position[2]=joystick.width/4
            heartcard2.position[3]=joystick.height-joystick.height/4+10
                    
        else:
            if heartcardslp2==0:
                heartcard2.position[0]-=2
                heartcard2.position[2]-=2
            else:
                if heartcard2.sleep==0:
                    heartcard2.position[0]=joystick.width/4-10
                    heartcard2.position[2]=joystick.width/4+14
                    heartcardslp2=0
                    heartcard2.sleep=20
                else:
                    heartcard2.sleep-=1
        if heartcard.position[3]==joystick.width/2+10:
            heartcardslp=1
            heartcard.position[0]=joystick.width/4-8
            heartcard.position[1]=joystick.height/4-1
            heartcard.position[2]=joystick.width/4+8
            heartcard.position[3]=joystick.height/4
                    
        else:
            if heartcardslp==0:
                heartcard.position[1]+=2
                heartcard.position[3]+=2
            else:
                if heartcard.sleep==0:
                    heartcard.position[1]=joystick.height/4-10
                    heartcard.position[3]=joystick.height/4+14
                    heartcardslp=0
                    heartcard.sleep=20
                else:
                    heartcard.sleep-=1
    
    if (bottle.notup(bottle.position,my_circle.position) or bottle.notdown(bottle.position,my_circle.position) or bottle.notleft(bottle.position,my_circle.position) or bottle.notright(bottle.position,my_circle.position)) and bottleeat==0:  
        bottleeat=1   
        my_circle.position[0]+=8
        my_circle.position[1]+=12
        my_circle.position[2]-=8
        my_circle.position[3]-=12
        relice = relice.resize(8, 12)
        lelice = lelice.resize(8, 12)
    if (mushroom.notup(mushroom.position,my_circle.position) or mushroom.notdown(mushroom.position,my_circle.position) or mushroom.notleft(mushroom.position,my_circle.position) or mushroom.notright(mushroom.position,my_circle.position)) and mushroomeat==0:  
        mushroomeat=1   
        my_circle.position[0]-=8
        my_circle.position[1]-=12
        my_circle.position[2]+=8
        my_circle.position[3]+=12
        relice = relice.resize(16, 24)
        lelice = lelice.resize(16, 24)
    if (hole1.notup(hole1.position,my_circle.position) or hole1.notdown(hole1.position,my_circle.position) or hole1.notleft(hole1.position,my_circle.position) or hole1.notright(hole1.position,my_circle.position)):  
        break
        
    #그리는 순서가 중요합니다. 배경을 먼저 깔고 위에 그림을 그리고 싶었는데 그림을 그려놓고 배경으로 덮는 결과로 될 수 있습니다.
    my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
    my_image.paste(iground,(0,0))
    my_draw.rectangle(tuple(heartcard.position), fill = (0, 0, 0,0))
    if heartcardslp==0:
        my_image.paste(iheartcard,(int(heartcard.position[0]),int(heartcard.position[1])))
    my_draw.rectangle(tuple(heartcard2.position), fill = (0, 0, 0,0))
    if heartcardslp2==0:
        my_image.paste(iheartcard2,(int(heartcard2.position[0]),int(heartcard2.position[1])))
    if bottleeat==0:
        my_draw.rectangle(tuple(bottle.position), fill = (0, 0, 0,0))
        my_image.paste(ibottle,(205, 73),mask=ibottle)
    if mushroomeat==0:
        my_draw.rectangle(tuple(mushroom.position), fill = (0, 0, 0,0))
        my_image.paste(imushroom,(10, 205),mask=imushroom)
    my_draw.rectangle(tuple(miro1.position), fill = (0, 0, 0,0))
    my_image.paste(imiro1,(0,50))
    my_draw.rectangle(tuple(miro2.position), fill = (0, 0, 0,0))
    my_image.paste(imiro2,(110, 0))
    my_draw.rectangle(tuple(miro3.position), fill = (0, 0, 0,0))
    my_image.paste(imiro3,(170, 50))
    my_draw.rectangle(tuple(miro4.position),fill = (0, 0, 0,0))
    my_image.paste(imiro4,(10, 110))
    my_draw.rectangle(tuple(miro5.position), fill = (0, 0, 0,0))
    my_image.paste(imiro5,(50, 175))
    my_draw.rectangle(tuple(miro6.position), fill = (0, 0, 0,0))
    my_draw.ellipse(tuple(hole1.position),fill = (0, 255,0,100))
    my_image.paste(imiro6,(50, 175))
    if delice==0:
        my_draw.ellipse(tuple(my_circle.position),fill = (0, 0,0,0))
        my_image.paste(relice,(int(my_circle.position[0]),int(my_circle.position[1])),mask=relice)
    elif delice==1:
        my_draw.ellipse(tuple(my_circle.position),fill = (0, 0, 0,0))
        my_image.paste(lelice,(int(my_circle.position[0]),int(my_circle.position[1])),mask=lelice)
        
    if (lifenum==3):
        my_image.paste(ilife,(135, 5))
        my_image.paste(ilife,(100, 5))
        my_image.paste(ilife,(65, 5))
    elif (lifenum==2):
        my_image.paste(ilife,(135, 5))
        my_image.paste(ilife,(100, 5))
    elif (lifenum==1):
        my_image.paste(ilife,(135, 5))
    elif (lifenum==0):
        break
        
        
    
    '''
    for enemy in enemys_list:
        if enemy.state != 'die':
            my_draw.ellipse(tuple(enemy.position), outline = enemy.outline, fill = (255, 0, 0))

    for bullet in bullets:
        if bullet.state != 'hit':
            my_draw.rectangle(tuple(bullet.position), outline = bullet.outline, fill = (0, 0, 255))
    '''

    #좌표는 동그라미의 왼쪽 위, 오른쪽 아래 점 (x1, y1, x2, y2)
    joystick.disp.image(my_image)
    #11/19 4:24 수정

In [ ]:
# 잔상이 남지 않는 코드 & 대각선 이동 가능
my_circle = Character(joystick.width, joystick.height)
miro1 = Miro1(joystick.width, joystick.height)
miro2 = Miro2(joystick.width, joystick.height)
miro3 = Miro3(joystick.width, joystick.height)
miro4 = Miro4(joystick.width, joystick.height)
miro5 = Miro5(joystick.width, joystick.height)
miro6 = Miro6(joystick.width, joystick.height)
heartcard = heartCard(joystick.width, joystick.height)
heartcard2 = heartCard2(joystick.width, joystick.height)
bottle = Bottle(joystick.width, joystick.height)
mushroom = Mushroom(joystick.width, joystick.height)
hole1 = Hole1(joystick.width, joystick.height)
'''
enemy_1 = Enemy((50, 50))
enemy_2 = Enemy((200, 200))
enemy_3 = Enemy((150, 50))

enemys_list = [enemy_1, enemy_2, enemy_3]


bullets = []
'''
heartcardslp=0
heartcardslp2=0
heartcrash=30
heartcrash2=30
bottleeat=0
mushroomeat=0
delice=0
while True:
    command = {'move': False, 'up_pressed': False , 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
    
    if not joystick.button_U.value:  # up pressed
        if miro1.notup(miro1.position, my_circle.position) and miro2.notup(miro2.position, my_circle.position) and miro4.notup(miro4.position, my_circle.position) and miro5.notup(miro5.position, my_circle.position):
            command['up_pressed'] = True
            command['move'] = True
            
    
    if not joystick.button_D.value:  # down pressed
        if miro1.notdown(miro1.position, my_circle.position) and miro3.notdown(miro3.position, my_circle.position) and miro4.notdown(miro4.position, my_circle.position) and miro5.notdown(miro5.position, my_circle.position) and miro6.notdown(miro6.position, my_circle.position):
            command['down_pressed'] = True
            command['move'] = True

    if not joystick.button_L.value: 
        delice=1
        if miro1.notleft(miro1.position, my_circle.position) and miro2.notleft(miro2.position, my_circle.position) and miro3.notleft(miro3.position, my_circle.position) and miro5.notleft(miro5.position, my_circle.position) and miro6.notleft(miro6.position, my_circle.position):
            command['left_pressed'] = True
            command['move'] = True
            

    if not joystick.button_R.value: 
        delice=0
        if miro2.notright(miro2.position, my_circle.position) and miro3.notright(miro3.position, my_circle.position) and miro4.notright(miro4.position, my_circle.position) and miro5.notright(miro5.position, my_circle.position) and miro6.notright(miro6.position, my_circle.position):
            command['right_pressed'] = True
            command['move'] = True
            
    '''
    if not joystick.button_A.value: # A pressed
        bullet = Bullet(my_circle.center, command)
        bullets.append(bullet)
    '''
    
        
    if (heartcard.notup(heartcard.position,my_circle.position) or heartcard.notdown(heartcard.position,my_circle.position) or heartcard.notleft(heartcard.position,my_circle.position) or heartcard.notright(heartcard.position,my_circle.position)) and heartcrash!=0:
        heartcrash-=1
        if heartcrash==29:
            lifenum-=1
    elif (heartcard2.notup(heartcard2.position,my_circle.position) or heartcard2.notdown(heartcard2.position,my_circle.position) or heartcard2.notleft(heartcard2.position,my_circle.position) or heartcard2.notright(heartcard2.position,my_circle.position)) and heartcrash2!=0:
        heartcrash2-=1
        if heartcrash2==29:
            lifenum-=1
        if heartcard.position[3]==joystick.width/2+10:
            heartcardslp=1
            heartcard.position[0]=joystick.width/4-8
            heartcard.position[1]=joystick.height/4-1
            heartcard.position[2]=joystick.width/4+8
            heartcard.position[3]=joystick.height/4
                    
        else:
            if heartcardslp==0:
                heartcard.position[1]+=2
                heartcard.position[3]+=2
            else:
                if heartcard.sleep==0:
                    heartcard.position[1]=joystick.height/4-10
                    heartcard.position[3]=joystick.height/4+14
                    heartcardslp=0
                    heartcard.sleep=20
                else:
                    heartcard.sleep-=1
    else:
        my_circle.move(command)
        
        if heartcard2.position[2]==0:
            heartcardslp2=1
            heartcard2.position[0]=joystick.width/4-1
            heartcard2.position[1]=joystick.height-joystick.height/4-6
            heartcard2.position[2]=joystick.width/4
            heartcard2.position[3]=joystick.height-joystick.height/4+10
                    
        else:
            if heartcardslp2==0:
                heartcard2.position[0]-=2
                heartcard2.position[2]-=2
            else:
                if heartcard2.sleep==0:
                    heartcard2.position[0]=joystick.width/4-10
                    heartcard2.position[2]=joystick.width/4+14
                    heartcardslp2=0
                    heartcard2.sleep=20
                else:
                    heartcard2.sleep-=1
        if heartcard.position[3]==joystick.width/2+10:
            heartcardslp=1
            heartcard.position[0]=joystick.width/4-8
            heartcard.position[1]=joystick.height/4-1
            heartcard.position[2]=joystick.width/4+8
            heartcard.position[3]=joystick.height/4
                    
        else:
            if heartcardslp==0:
                heartcard.position[1]+=2
                heartcard.position[3]+=2
            else:
                if heartcard.sleep==0:
                    heartcard.position[1]=joystick.height/4-10
                    heartcard.position[3]=joystick.height/4+14
                    heartcardslp=0
                    heartcard.sleep=20
                else:
                    heartcard.sleep-=1
    
    if (bottle.notup(bottle.position,my_circle.position) or bottle.notdown(bottle.position,my_circle.position) or bottle.notleft(bottle.position,my_circle.position) or bottle.notright(bottle.position,my_circle.position)) and bottleeat==0:  
        bottleeat=1   
        my_circle.position[0]+=8
        my_circle.position[1]+=12
        my_circle.position[2]-=8
        my_circle.position[3]-=12
        relice = relice.resize(8, 12)
        lelice = lelice.resize(8, 12)
    if (mushroom.notup(mushroom.position,my_circle.position) or mushroom.notdown(mushroom.position,my_circle.position) or mushroom.notleft(mushroom.position,my_circle.position) or mushroom.notright(mushroom.position,my_circle.position)) and mushroomeat==0:  
        mushroomeat=1   
        my_circle.position[0]-=8
        my_circle.position[1]-=12
        my_circle.position[2]+=8
        my_circle.position[3]+=12
        relice = relice.resize(16, 24)
        lelice = lelice.resize(16, 24)
    if (hole1.notup(hole1.position,my_circle.position) or hole1.notdown(hole1.position,my_circle.position) or hole1.notleft(hole1.position,my_circle.position) or hole1.notright(hole1.position,my_circle.position)):  
        break
        
    #그리는 순서가 중요합니다. 배경을 먼저 깔고 위에 그림을 그리고 싶었는데 그림을 그려놓고 배경으로 덮는 결과로 될 수 있습니다.
    my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
    my_image.paste(iground,(0,0))
    my_draw.rectangle(tuple(heartcard.position), fill = (0, 0, 0,0))
    if heartcardslp==0:
        my_image.paste(iheartcard,(int(heartcard.position[0]),int(heartcard.position[1])))
    my_draw.rectangle(tuple(heartcard2.position), fill = (0, 0, 0,0))
    if heartcardslp2==0:
        my_image.paste(iheartcard2,(int(heartcard2.position[0]),int(heartcard2.position[1])))
    if bottleeat==0:
        my_draw.rectangle(tuple(bottle.position), fill = (0, 0, 0,0))
        my_image.paste(ibottle,(205, 73),mask=ibottle)
    if mushroomeat==0:
        my_draw.rectangle(tuple(mushroom.position), fill = (0, 0, 0,0))
        my_image.paste(imushroom,(10, 205),mask=imushroom)
    my_draw.rectangle(tuple(miro1.position), fill = (0, 0, 0,0))
    my_image.paste(imiro1,(0,50))
    my_draw.rectangle(tuple(miro2.position), fill = (0, 0, 0,0))
    my_image.paste(imiro2,(110, 0))
    my_draw.rectangle(tuple(miro3.position), fill = (0, 0, 0,0))
    my_image.paste(imiro3,(170, 50))
    my_draw.rectangle(tuple(miro4.position),fill = (0, 0, 0,0))
    my_image.paste(imiro4,(10, 110))
    my_draw.rectangle(tuple(miro5.position), fill = (0, 0, 0,0))
    my_image.paste(imiro5,(50, 175))
    my_draw.rectangle(tuple(miro6.position), fill = (0, 0, 0,0))
    my_draw.ellipse(tuple(hole1.position),fill = (0, 255,0,100))
    my_image.paste(imiro6,(50, 175))
    if delice==0:
        my_draw.ellipse(tuple(my_circle.position),fill = (0, 0,0,0))
        my_image.paste(relice,(int(my_circle.position[0]),int(my_circle.position[1])),mask=relice)
    elif delice==1:
        my_draw.ellipse(tuple(my_circle.position),fill = (0, 0, 0,0))
        my_image.paste(lelice,(int(my_circle.position[0]),int(my_circle.position[1])),mask=lelice)
        
    if (lifenum==3):
        my_image.paste(ilife,(135, 5))
        my_image.paste(ilife,(100, 5))
        my_image.paste(ilife,(65, 5))
    elif (lifenum==2):
        my_image.paste(ilife,(135, 5))
        my_image.paste(ilife,(100, 5))
    elif (lifenum==1):
        my_image.paste(ilife,(135, 5))
    elif (lifenum==0):
        break
        
        
    
    '''
    for enemy in enemys_list:
        if enemy.state != 'die':
            my_draw.ellipse(tuple(enemy.position), outline = enemy.outline, fill = (255, 0, 0))

    for bullet in bullets:
        if bullet.state != 'hit':
            my_draw.rectangle(tuple(bullet.position), outline = bullet.outline, fill = (0, 0, 255))
    '''

    #좌표는 동그라미의 왼쪽 위, 오른쪽 아래 점 (x1, y1, x2, y2)
    joystick.disp.image(my_image)

In [ ]:
# 잔상이 남지 않는 코드 & 대각선 이동 가능
my_circle = Character(joystick.width, joystick.height)
miro1 = Miro1(joystick.width, joystick.height)
miro2 = Miro2(joystick.width, joystick.height)
miro3 = Miro3(joystick.width, joystick.height)
miro4 = Miro4(joystick.width, joystick.height)
miro5 = Miro5(joystick.width, joystick.height)
miro6 = Miro6(joystick.width, joystick.height)
heartcard = heartCard(joystick.width, joystick.height)
heartcard2 = heartCard2(joystick.width, joystick.height)
bottle = Bottle(joystick.width, joystick.height)
mushroom = Mushroom(joystick.width, joystick.height)
hole1 = Hole1(joystick.width, joystick.height)
'''
enemy_1 = Enemy((50, 50))
enemy_2 = Enemy((200, 200))
enemy_3 = Enemy((150, 50))

enemys_list = [enemy_1, enemy_2, enemy_3]


bullets = []
'''
heartcardslp=0
heartcardslp2=0
heartcrash=30
heartcrash2=30
bottleeat=0
mushroomeat=0
delice=0
while True:
    command = {'move': False, 'up_pressed': False , 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
    
    if not joystick.button_U.value:  # up pressed
        if miro1.notup(miro1.position, my_circle.position) and miro2.notup(miro2.position, my_circle.position) and miro4.notup(miro4.position, my_circle.position) and miro5.notup(miro5.position, my_circle.position):
            command['up_pressed'] = True
            command['move'] = True
            
    
    if not joystick.button_D.value:  # down pressed
        if miro1.notdown(miro1.position, my_circle.position) and miro3.notdown(miro3.position, my_circle.position) and miro4.notdown(miro4.position, my_circle.position) and miro5.notdown(miro5.position, my_circle.position) and miro6.notdown(miro6.position, my_circle.position):
            command['down_pressed'] = True
            command['move'] = True

    if not joystick.button_L.value: 
        delice=1
        if miro1.notleft(miro1.position, my_circle.position) and miro2.notleft(miro2.position, my_circle.position) and miro3.notleft(miro3.position, my_circle.position) and miro5.notleft(miro5.position, my_circle.position) and miro6.notleft(miro6.position, my_circle.position):
            command['left_pressed'] = True
            command['move'] = True
            

    if not joystick.button_R.value: 
        delice=0
        if miro2.notright(miro2.position, my_circle.position) and miro3.notright(miro3.position, my_circle.position) and miro4.notright(miro4.position, my_circle.position) and miro5.notright(miro5.position, my_circle.position) and miro6.notright(miro6.position, my_circle.position):
            command['right_pressed'] = True
            command['move'] = True
            
    '''
    if not joystick.button_A.value: # A pressed
        bullet = Bullet(my_circle.center, command)
        bullets.append(bullet)
    '''
    
        
    if (heartcard.notup(heartcard.position,my_circle.position) or heartcard.notdown(heartcard.position,my_circle.position) or heartcard.notleft(heartcard.position,my_circle.position) or heartcard.notright(heartcard.position,my_circle.position)) and heartcrash!=0:
        heartcrash-=1
        if heartcrash==29:
            lifenum-=1
    elif (heartcard2.notup(heartcard2.position,my_circle.position) or heartcard2.notdown(heartcard2.position,my_circle.position) or heartcard2.notleft(heartcard2.position,my_circle.position) or heartcard2.notright(heartcard2.position,my_circle.position)) and heartcrash2!=0:
        heartcrash2-=1
        if heartcrash2==29:
            lifenum-=1
        if heartcard.position[3]==joystick.width/2+10:
            heartcardslp=1
            heartcard.position[0]=joystick.width/4-8
            heartcard.position[1]=joystick.height/4-1
            heartcard.position[2]=joystick.width/4+8
            heartcard.position[3]=joystick.height/4
                    
        else:
            if heartcardslp==0:
                heartcard.position[1]+=2
                heartcard.position[3]+=2
            else:
                if heartcard.sleep==0:
                    heartcard.position[1]=joystick.height/4-10
                    heartcard.position[3]=joystick.height/4+14
                    heartcardslp=0
                    heartcard.sleep=20
                else:
                    heartcard.sleep-=1
    else:
        my_circle.move(command)
        
        if heartcard2.position[2]==0:
            heartcardslp2=1
            heartcard2.position[0]=joystick.width/4-1
            heartcard2.position[1]=joystick.height-joystick.height/4-6
            heartcard2.position[2]=joystick.width/4
            heartcard2.position[3]=joystick.height-joystick.height/4+10
                    
        else:
            if heartcardslp2==0:
                heartcard2.position[0]-=2
                heartcard2.position[2]-=2
            else:
                if heartcard2.sleep==0:
                    heartcard2.position[0]=joystick.width/4-10
                    heartcard2.position[2]=joystick.width/4+14
                    heartcardslp2=0
                    heartcard2.sleep=20
                else:
                    heartcard2.sleep-=1
        if heartcard.position[3]==joystick.width/2+10:
            heartcardslp=1
            heartcard.position[0]=joystick.width/4-8
            heartcard.position[1]=joystick.height/4-1
            heartcard.position[2]=joystick.width/4+8
            heartcard.position[3]=joystick.height/4
                    
        else:
            if heartcardslp==0:
                heartcard.position[1]+=2
                heartcard.position[3]+=2
            else:
                if heartcard.sleep==0:
                    heartcard.position[1]=joystick.height/4-10
                    heartcard.position[3]=joystick.height/4+14
                    heartcardslp=0
                    heartcard.sleep=20
                else:
                    heartcard.sleep-=1
    
    if (bottle.notup(bottle.position,my_circle.position) or bottle.notdown(bottle.position,my_circle.position) or bottle.notleft(bottle.position,my_circle.position) or bottle.notright(bottle.position,my_circle.position)) and bottleeat==0:  
        bottleeat=1   
        my_circle.position[0]+=8
        my_circle.position[1]+=12
        my_circle.position[2]-=8
        my_circle.position[3]-=12
        relice = relice.resize(8, 12)
        lelice = lelice.resize(8, 12)
    if (mushroom.notup(mushroom.position,my_circle.position) or mushroom.notdown(mushroom.position,my_circle.position) or mushroom.notleft(mushroom.position,my_circle.position) or mushroom.notright(mushroom.position,my_circle.position)) and mushroomeat==0:  
        mushroomeat=1   
        my_circle.position[0]-=8
        my_circle.position[1]-=12
        my_circle.position[2]+=8
        my_circle.position[3]+=12
        relice = relice.resize(16, 24)
        lelice = lelice.resize(16, 24)
    if (hole1.notup(hole1.position,my_circle.position) or hole1.notdown(hole1.position,my_circle.position) or hole1.notleft(hole1.position,my_circle.position) or hole1.notright(hole1.position,my_circle.position)):  
        break
        
    #그리는 순서가 중요합니다. 배경을 먼저 깔고 위에 그림을 그리고 싶었는데 그림을 그려놓고 배경으로 덮는 결과로 될 수 있습니다.
    my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
    my_image.paste(iground,(0,0))
    my_draw.rectangle(tuple(heartcard.position), fill = (0, 0, 0,0))
    if heartcardslp==0:
        my_image.paste(iheartcard,(int(heartcard.position[0]),int(heartcard.position[1])))
    my_draw.rectangle(tuple(heartcard2.position), fill = (0, 0, 0,0))
    if heartcardslp2==0:
        my_image.paste(iheartcard2,(int(heartcard2.position[0]),int(heartcard2.position[1])))
    if bottleeat==0:
        my_draw.rectangle(tuple(bottle.position), fill = (0, 0, 0,0))
        my_image.paste(ibottle,(205, 73),mask=ibottle)
    if mushroomeat==0:
        my_draw.rectangle(tuple(mushroom.position), fill = (0, 0, 0,0))
        my_image.paste(imushroom,(10, 205),mask=imushroom)
    my_draw.rectangle(tuple(miro1.position), fill = (0, 0, 0,0))
    my_image.paste(imiro1,(0,50))
    my_draw.rectangle(tuple(miro2.position), fill = (0, 0, 0,0))
    my_image.paste(imiro2,(110, 0))
    my_draw.rectangle(tuple(miro3.position), fill = (0, 0, 0,0))
    my_image.paste(imiro3,(170, 50))
    my_draw.rectangle(tuple(miro4.position),fill = (0, 0, 0,0))
    my_image.paste(imiro4,(10, 110))
    my_draw.rectangle(tuple(miro5.position), fill = (0, 0, 0,0))
    my_image.paste(imiro5,(50, 175))
    my_draw.rectangle(tuple(miro6.position), fill = (0, 0, 0,0))
    my_draw.ellipse(tuple(hole1.position),fill = (0, 255,0,100))
    my_image.paste(imiro6,(50, 175))
    if delice==0:
        my_draw.ellipse(tuple(my_circle.position),fill = (0, 0,0,0))
        my_image.paste(relice,(int(my_circle.position[0]),int(my_circle.position[1])),mask=relice)
    elif delice==1:
        my_draw.ellipse(tuple(my_circle.position),fill = (0, 0, 0,0))
        my_image.paste(lelice,(int(my_circle.position[0]),int(my_circle.position[1])),mask=lelice)
        
    if (lifenum==3):
        my_image.paste(ilife,(135, 5))
        my_image.paste(ilife,(100, 5))
        my_image.paste(ilife,(65, 5))
    elif (lifenum==2):
        my_image.paste(ilife,(135, 5))
        my_image.paste(ilife,(100, 5))
    elif (lifenum==1):
        my_image.paste(ilife,(135, 5))
    elif (lifenum==0):
        break
        
        
    
    '''
    for enemy in enemys_list:
        if enemy.state != 'die':
            my_draw.ellipse(tuple(enemy.position), outline = enemy.outline, fill = (255, 0, 0))

    for bullet in bullets:
        if bullet.state != 'hit':
            my_draw.rectangle(tuple(bullet.position), outline = bullet.outline, fill = (0, 0, 255))
    '''

    #좌표는 동그라미의 왼쪽 위, 오른쪽 아래 점 (x1, y1, x2, y2)
    joystick.disp.image(my_image)